---
# Cairo University Faculty of Engineering
## Machine Learning 
## Assignment 7

---
Please write your full name here
- **Name** : "Yousef Adham Mahmoud Awad Mahmoud"

### Section 1 - Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline


### Section 2 - Reading and preprocessing the dataset

In [2]:
# Converting the csv files to panadas dataframes and storing them in variables
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# Since we don't need the genres, and the timestamp in Collaborative Filtering, we will drop them.
movies_df = movies_df.drop([ 'genres'], axis=1)
ratings_df = ratings_df.drop(['timestamp'], axis = 1)


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [7]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [8]:
# Checking the max rating
max_value = ratings_df['rating'].max()
print(max_value)
min_value = ratings_df['rating'].min()
print(min_value)

5.0
0.5


In [9]:
# Merging the 2 datasets together
users_ratings_df = pd.merge(movies_df, ratings_df, on='movieId')
users_ratings_df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [10]:
users_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int64  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int64  
 3   rating   100836 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [11]:
# Pivoting the users to the rows and making the columns the name of the movies with the cells being the rating given by the user
users_ratings_df = users_ratings_df.pivot_table(index='userId', columns='title', values='rating', sort=False)
users_ratings_df.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Sorting the userId
users_ratings_df = users_ratings_df.sort_values(by='userId')
users_ratings_df.head()


title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Filtering the users to the first 200 user, and 200 movies
filtered_users = users_ratings_df.iloc[:200]
filtered_users = filtered_users.iloc[:, :200]
filtered_users

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Clerks (1994),Don Juan DeMarco (1995),Disclosure (1994),Drop Zone (1994),Destiny Turns on the Radio (1995),Death and the Maiden (1994),Dolores Claiborne (1995),Dumb & Dumber (Dumb and Dumber) (1994),Eat Drink Man Woman (Yin shi nan nu) (1994),Exotica (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:

# Identify empty rows
empty_rows = filtered_users[filtered_users.isnull().all(axis=1)]

# Drop empty rows from the DataFrame
filtered_users.drop(empty_rows.index, inplace=True)

filtered_users.fillna(0, inplace=True)

In [15]:
filtered_users

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Clerks (1994),Don Juan DeMarco (1995),Disclosure (1994),Drop Zone (1994),Destiny Turns on the Radio (1995),Death and the Maiden (1994),Dolores Claiborne (1995),Dumb & Dumber (Dumb and Dumber) (1994),Eat Drink Man Woman (Yin shi nan nu) (1994),Exotica (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,3.0,4.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Section 3 - Calculating the cosine similarity matrix

In [16]:

# Calculate cosine similarity
cosine_sim = cosine_similarity(filtered_users.T)
cosine_sim_df = pd.DataFrame(cosine_sim, columns=filtered_users.columns, index=filtered_users.columns)

In [17]:
cosine_sim_df.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Clerks (1994),Don Juan DeMarco (1995),Disclosure (1994),Drop Zone (1994),Destiny Turns on the Radio (1995),Death and the Maiden (1994),Dolores Claiborne (1995),Dumb & Dumber (Dumb and Dumber) (1994),Eat Drink Man Woman (Yin shi nan nu) (1994),Exotica (1994)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.364634,0.284825,0.000000,0.279775,0.327928,0.270701,0.128765,0.195017,0.352899,...,0.363093,0.211791,0.160623,0.088024,0.107733,0.104516,0.140704,0.360625,0.087884,0.0
Jumanji (1995),0.364634,1.000000,0.278446,0.102937,0.277072,0.278936,0.240488,0.130496,0.000000,0.342590,...,0.202565,0.085855,0.132766,0.114888,0.000000,0.000000,0.191316,0.352531,0.000000,0.0
Grumpier Old Men (1995),0.284825,0.278446,1.000000,0.150278,0.391915,0.341700,0.592157,0.406427,0.288534,0.231410,...,0.284289,0.200545,0.347200,0.197325,0.000000,0.000000,0.244260,0.485324,0.000000,0.0
Waiting to Exhale (1995),0.000000,0.102937,0.150278,1.000000,0.239259,0.176604,0.297614,0.253546,0.000000,0.129040,...,0.108721,0.382276,0.280386,0.000000,0.000000,0.000000,0.369406,0.132357,0.257513,0.0
Father of the Bride Part II (1995),0.279775,0.277072,0.391915,0.239259,1.000000,0.256166,0.539392,0.404422,0.000000,0.275293,...,0.150873,0.295177,0.380707,0.070687,0.000000,0.000000,0.287247,0.296883,0.092419,0.0


### Section 4 - finding the top 10 similar movies to toy story, and waiting to 

In [18]:
# Getting the top 10 movies similar to toy story in this dataset
toy_story_row = cosine_sim_df.loc['Toy Story (1995)']
sorted_row = toy_story_row.sort_values(ascending=False)
sorted_row = sorted_row[1:]
# Retrieve the top 10 values from the modified sorted row
top_10_values = sorted_row.head(10)

# Get the indices and values of these top 10 cells
top_10_indices = top_10_values.index.tolist()
top_10_cell_values = top_10_values.values.tolist()

# Display the indices and values
for index, value in zip(top_10_indices, top_10_cell_values):
    print("Title:", index, "Similarity:", round(value,2))

Title: Braveheart (1995) Similarity: 0.51
Title: Apollo 13 (1995) Similarity: 0.49
Title: Usual Suspects, The (1995) Similarity: 0.46
Title: Happy Gilmore (1996) Similarity: 0.45
Title: Die Hard: With a Vengeance (1995) Similarity: 0.44
Title: Seven (a.k.a. Se7en) (1995) Similarity: 0.43
Title: Babe (1995) Similarity: 0.42
Title: Waterworld (1995) Similarity: 0.42
Title: Batman Forever (1995) Similarity: 0.41
Title: Taxi Driver (1976) Similarity: 0.4


In [19]:
exhale_row = cosine_sim_df.loc['Waiting to Exhale (1995)']
sorted_row = exhale_row.sort_values(ascending=False)
sorted_row = sorted_row[1:]
# Retrieve the top 10 values from the modified sorted row
top_10_values = sorted_row.head(10)

# Get the indices and values of these top 10 cells
top_10_indices = top_10_values.index.tolist()
top_10_cell_values = top_10_values.values.tolist()

# Display the indices and values
for index, value in zip(top_10_indices, top_10_cell_values):
    print("Title:", index, "Similarity:", round(value,2))

Title: Before and After (1996) Similarity: 0.6
Title: Mighty Morphin Power Rangers: The Movie (1995) Similarity: 0.52
Title: Three Wishes (1995) Similarity: 0.5
Title: Reckless (1995) Similarity: 0.5
Title: Now and Then (1995) Similarity: 0.5
Title: Bushwhacked (1995) Similarity: 0.5
Title: Cure, The (1995) Similarity: 0.5
Title: Mad Love (1995) Similarity: 0.5
Title: Georgia (1995) Similarity: 0.5
Title: Before the Rain (Pred dozhdot) (1994) Similarity: 0.5


### Section 5 - Using the calculate similarity matrix to recommend 3 films to userID 200

In [20]:
user_200 = filtered_users.loc[200]

user_200_movies = []
for column_name, value in user_200.iteritems():
    if value > 0:
        user_200_movies.append((column_name, value))
len(user_200_movies)

<ipython-input-20-3aad3c94f4b1>:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, value in user_200.iteritems():


13

In [21]:
# Function that returns the similarity of a watched movie with all the other movies, excluding itself
def similarity_function(movie, rating, watched_movies):
  
  # Less weight given for movies with a rating lower than 3 
  similarity_score = cosine_sim_df[movie] * (rating - 3)
  similarity_score = similarity_score.sort_values(ascending=False)

  similarity_score = similarity_score[~similarity_score.index.isin(watched_movies)]  # Exclude watched movies

  return similarity_score

In [22]:
# Creating the dataframe that will store the recommended movies
similar_movies = pd.DataFrame()

# Creating a list with the already watched movies by the user
watched_movies = [movie for movie, _ in user_200_movies]

# Looping over every movie and calculating its similarity
for movie, rating in user_200_movies:
  similar_movies = similar_movies.append(similarity_function(movie, rating, watched_movies), ignore_index =True)


similar_movies.head()


<ipython-input-22-57a74aa2a522>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(similarity_function(movie, rating, watched_movies), ignore_index =True)
<ipython-input-22-57a74aa2a522>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(similarity_function(movie, rating, watched_movies), ignore_index =True)
<ipython-input-22-57a74aa2a522>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(similarity_function(movie, rating, watched_movies), ignore_index =True)
<ipython-input-22-57a74aa2a522>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

title,"Usual Suspects, The (1995)",Waterworld (1995),Batman Forever (1995),Taxi Driver (1976),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Birdcage, The (1996)",Jumanji (1995),Clerks (1994),Dumb & Dumber (Dumb and Dumber) (1994),"Net, The (1995)",...,Party Girl (1995),Catwalk (1996),Reckless (1995),Safe (1995),"Scarlet Letter, The (1995)",Nobody Loves Me (Keiner liebt mich) (1994),Unforgettable (1996),Something to Talk About (1995),In the Bleak Midwinter (1995),Exotica (1994)
0,0.229969,0.209769,0.203604,0.201206,0.200646,0.189522,0.182317,0.181546,0.180313,0.176120,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,0.173593,0.380726,0.239259,0.125042,0.220042,0.421279,0.277072,0.150873,0.296883,0.367620,...,0.0,0.0,0.299074,0.000000,0.347750,0.0,0.0,0.235606,0.0,0.000000
2,0.413964,0.603673,0.828494,0.204261,0.381495,0.420090,0.513885,0.249021,0.617699,0.754350,...,0.0,0.0,0.193560,0.115417,0.230833,0.0,0.0,0.508278,0.0,0.132781
3,0.152987,0.196163,0.169112,0.126558,0.186176,0.130070,0.172293,0.178769,0.242196,0.202375,...,0.0,0.0,0.069379,0.000000,0.062055,0.0,0.0,0.123830,0.0,0.000000
4,-0.178955,-0.207728,-0.237399,-0.114636,-0.185824,-0.130908,-0.230035,-0.139401,-0.184695,-0.256842,...,-0.0,-0.0,-0.080895,-0.000000,-0.117576,-0.0,-0.0,-0.104553,-0.0,-0.031215


In [23]:
# Getting the top 3 recommended movies
top_3_indices = similar_movies.sum().sort_values(ascending=False).head(3).index


In [24]:
# printing the top 3 recommended movies
print("Recommended movies:")
for movie in top_3_indices:
  print(movie)

Recommended movies:
Dumb & Dumber (Dumb and Dumber) (1994)
Batman Forever (1995)
Net, The (1995)
